In [5]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
import io
import matplotlib.pyplot as plt

# Widget de carga de archivos
uploader = widgets.FileUpload(
    accept='.csv,.xlsx,.xls',  # Formatos aceptados
    multiple=False,  # Solo un archivo
    description='Subir archivo'
)

# Elementos de la interfaz
output = widgets.Output()
file_info = widgets.HTML(value="<i>Esperando archivo...</i>")
preview_btn = widgets.Button(description="Previsualizar", disabled=True)
plot_btn = widgets.Button(description="Graficar", disabled=True)

# Función adaptada para la nueva estructura de uploader.value
def handle_upload(change):
    if not uploader.value:
        file_info.value = "<i>No se ha subido ningún archivo</i>"
        preview_btn.disabled = True
        plot_btn.disabled = True
        return
    
    try:
        # Estructura actual: uploader.value es una tupla de diccionarios
        file_data = uploader.value[0]  # Primer archivo (aunque multiple=False)
        
        file_info.value = f"""
        <div style="background:#f0f0f0; padding:10px; border-radius:5px;">
        <b>Archivo:</b> {file_data['name']}<br>
        <b>Tamaño:</b> {len(file_data['content'])/1024:.1f} KB<br>
        <b>Tipo:</b> {file_data['type']}
        </div>
        """
        preview_btn.disabled = False
        plot_btn.disabled = False
        
    except Exception as e:
        file_info.value = f"<div style='color:red'>Error al procesar archivo: {str(e)}</div>"
        preview_btn.disabled = True
        plot_btn.disabled = True

# Asociar la función al evento de cambio
uploader.observe(handle_upload, names='value')

def show_preview(b):
    with output:
        clear_output()
        try:
            file_data = uploader.value[0]
            content = file_data['content']
            
            if file_data['name'].endswith('.csv'):
                df = pd.read_csv(io.BytesIO(content))
            else:  # Excel
                df = pd.read_excel(io.BytesIO(content))
                
            display(df.head(5))
            display(df.info())
            
        except Exception as e:
            print(f"Error al leer archivo: {str(e)}")

def generate_plot(b):
    with output:
        clear_output()
        try:
            file_data = uploader.value[0]
            content = file_data['content']
            
            if file_data['name'].endswith('.csv'):
                df = pd.read_csv(io.BytesIO(content))
            else:
                df = pd.read_excel(io.BytesIO(content))
            
            # Seleccionar automáticamente columnas numéricas
            numeric_cols = df.select_dtypes(include=['number']).columns
            
            if len(numeric_cols) >= 2:
                plt.figure(figsize=(10, 5))
                df.plot(x=numeric_cols[0], y=numeric_cols[1])
                plt.title(f"Gráfico de {numeric_cols[0]} vs {numeric_cols[1]}")
                plt.grid(True)
                plt.show()
            else:
                print("El archivo no contiene suficientes columnas numéricas para graficar")
                
        except Exception as e:
            print(f"Error al generar gráfico: {str(e)}")

# Asignar funciones a los botones
preview_btn.on_click(show_preview)
plot_btn.on_click(generate_plot)

# Mostrar la interfaz completa
display(widgets.VBox([
    widgets.HTML("<h2>Cargador de Archivos con Voilà</h2>"),
    uploader,
    file_info,
    widgets.HBox([preview_btn, plot_btn]),
    output
]))